In [113]:
from PIL import Image
import numpy as np



img = Image.open('/Users/taewan/Desktop/Project/img_to_text/pi.png')
img = img.convert('L')
img = img.resize((30,30), Image.ANTIALIAS)

pixel_data = img.getdata()
print(pixel_data)
pixels = np.array([pixel_data])
#print(pixelss)
print(pixels)
#for i in pixelss :
    #pixels.append(i)
#pixels.appen(pixels)
#print(pixels)

img_label = [[1,0,0,0,0,0,0,0,0,0]]

#pr



[[225 211 229 215 215 210 199 208 206 221 213 215 208 206 190 206 189 199
  213 214 212 205 211 212 222 216 209 235 210 220 220 187 199 173 211 202
  174 188 179 179 201 197 196 164 169 184 218 177 193 199 212 167 168 158
  171 199 172 167 186 202 239 229 249 245 246 241 248 227 236 234 236 237
  238 230 242 242 255 247 236 221 237 233 239 245 230 223 239 243 237 239
  243 244 237 241 237 228 234 241 241 239 238 241 239 254 243 242 237 227
  242 254 243 242 240 240 242 245 234 229 242 247 222 165 181 201 204 203
  170 177 181 189 192 162 162 173 190 192 171 206 198 181 172 182 183 158
  166 192 205 198 196 213 227 194 205 205 204 240 228 211 201 198 204 196
  191 203 215 231 209 241 230 201 208 212 215 217 214 209 225 236 210 233
  255 255 255 255 252 255 255 253 255 255 255 255 255 255 255 255 255 255
  255 254 255 255 255 255 255 255 255 250 255 255 209 192 203 196 205 193
  181 198 207 191 193 207 208 200 193 197 210 202 198 199 187 199 207 221
  205 194 195 227 191 198 191 193 176 

In [115]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.reset_default_graph()

np.random.seed(11111111)
tf.set_random_seed(11111111)

## first_layer

first_filter = 32

x = tf.placeholder(tf.float32, [None, 900])
image_x = tf.reshape(x, [-1,32,32,1])

#print(image_x)
# 필터 크기 : 5x5x1, 필터 수 = 32 / tf.truncated_normal : 임의의 수 지정
# 스트라이드 = 1,1,1,1 / 패딩 : 특징 유실 방지(이미지 크기 보존)
first_conv_w = tf.Variable(tf.truncated_normal([5,5,1,first_filter],stddev = 0.1))
first_conv_h = tf.nn.conv2d(image_x, first_conv_w, strides=[1,1,1,1], padding = 'SAME')

#print(first_conv_w)
#print(first_conv_h)
first_conv_b = tf.Variable(tf.constant(0.1, shape = [first_filter]))
first_result = tf.nn.relu(first_conv_h + first_conv_b)

#print(first_conv_b)
#print(first_result)

first_pool = tf.nn.max_pool(first_result, ksize=[1,2,2,1],
                           strides = [1,2,2,1], padding = 'SAME')

#print(first_pool)

## second_layer

second_filter = 64

second_conv_w = tf.Variable(tf.truncated_normal([5,5,first_filter,
                                                 second_filter], stddev = 0.1))
#print(second_conv_w)
second_conv_h = tf.nn.conv2d(first_pool, second_conv_w,
                            strides = [1,1,1,1], padding = 'SAME')

second_conv_b = tf.Variable(tf.constant(0.1, shape =[second_filter]))
print(second_conv_b)
second_result = tf.nn.relu(second_conv_h + second_conv_b)

second_pool = tf.nn.max_pool(second_result, ksize = [1,2,2,1],
                            strides = [1,2,2,1], padding = 'SAME')

second_pool_flat = tf.reshape(second_pool, [-1, 8*8*second_filter])
print(second_pool_flat)

num_vec = 8*8*second_filter
num_neuron = 1024

w2 = tf.Variable(tf.truncated_normal([num_vec, num_neuron]))
b2 = tf.Variable(tf.Variable(tf.constant(0.1, shape = [num_neuron])))
hidden2 = tf.nn.relu(tf.matmul(second_pool_flat,w2) +b2)

# dropout : 일부러 일부 노드 연결 끊음 -> 오버피팅 방지
# hidden2 : 앞의 네트워크에서 전달된 값으로 keep_prob에 연결 비율 넣음

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_neuron, 10]))
b0 = tf.Variable(tf.zeros([10]))
k = tf.matmul(hidden2_drop, w0) + b0
p = tf.nn.softmax(k)

#loss function : 크로스 엔트로피

t = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits =k,labels = t))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

correct_prediction = tf.equal(tf.argmax(p,1), tf.argmax(t,1))
accuarcy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

# start training
i = 0

for _ in range(1000) :

    i = i + 1

    

    sess.run(train_step,

             feed_dict={x:pixels, t:img_label, keep_prob:0.5})

    if i % 500 == 0:

        loss_vals, acc_vals = [], []

        for c in range(4):

            start = len(pixels) / 4 * c

            end = len(img_label) / 4 * (c+1)

            loss_val, acc_val = sess.run([loss, accuracy],

                feed_dict={x:pixels[start:end],

                           t:img_label[start:end],

                           keep_prob:1.0})

            loss_vals.append(loss_val)

            acc_vals.append(acc_val)

        loss_val = np.sum(loss_vals)

        acc_val = np.mean(acc_vals)

        print ('Step: %d, Loss: %f, Accuracy: %f'

               % (i, loss_val, acc_val))



saver.save(sess, 'cnn_session')

sess.close()



<tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>
Tensor("Reshape_1:0", shape=(?, 4096), dtype=float32)


/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


InvalidArgumentError: Input to reshape is a tensor with 900 values, but the requested shape requires a multiple of 1024
	 [[Node: Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_Placeholder_0_0, Reshape/shape)]]

Caused by op 'Reshape', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  F